In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import glob
import math
warnings.filterwarnings('ignore')

In [49]:
def preprocessing(input_path):
    all_input_list = sorted(glob.glob(input_path))
    train = pd.DataFrame()
    for datapath in all_input_list:
        data = pd.read_csv(datapath) # 
        data['obs_time'] = data.index % 24 # 시간통일
        df = abs(data)
        df.loc[(df['내부온도관측치'] > 40), '내부온도관측치'] = 40
        df.loc[(df['내부습도관측치'] > 81.269491), '내부습도관측치'] = 81.269491
        df.loc[(df['co2관측치'] > 670.600000), 'co2관측치'] = 670.600000
        df.loc[(df['ec관측치'] > 8), 'ec관측치'] = 8
        df.loc[(df['시간당분무량'] > 1345.100000), '시간당분무량'] = 1345.100000
        df.loc[(df['시간당백색광량'] > 18255.190000), '시간당백색광량'] = 18255.190000
        df.loc[(df['시간당적색광량'] > 5857.992000), '시간당적색광량'] = 5857.992000
        df.loc[(df['시간당청색광량'] > 3696.940000), '시간당청색광량'] = 3696.940000
        
        df.loc[(df['내부온도관측치'] < 20.665000), '내부온도관측치'] = 20.665000
        df.loc[(df['내부습도관측치'] < 51.512500), '내부습도관측치'] = 51.512500
        df.loc[(df['co2관측치']<  383.316667), 'co2관측치'] = 383.316667
        
        #df = df.drop(['시간당총광량','일간누적총광량'],axis=1)
        col_list = df.columns
        for i in range(0,len(col_list)):
            col = col_list[i]    
            if '누적' in col :
                df[col] = df.groupby((df.obs_time == 0).cumsum()).agg(col_list[i-1]).cumsum()            
            df.to_csv(datapath,index=False)
            train = pd.concat([train,df])
    print('finish!!')
    return train

In [46]:
traininput.describe(percentiles=[.05, .10, .95])

,DAT,obs_time,내부온도관측치,내부습도관측치,co2관측치,ec관측치,시간당분무량,일간누적분무량,시간당백색광량,일간누적백색광량,시간당적색광량,일간누적적색광량,시간당청색광량,일간누적청색광량
count,263424.000000,263424.0000,263424.000000,263424.000000,263424.000000,263424.000000,263424.000000,263424.000000,263424.000000,263424.000000,263424.000000,263424.000000,263424.000000,263424.000000
mean,13.500000,11.5000,26.027343,63.841437,490.311429,1.507566,510.800023,6288.332222,6322.804696,72932.655472,1572.639303,18202.752672,1243.250943,14140.019249
std,8.077763,6.9222,3.947493,11.502718,87.113233,1.041984,514.650914,6732.065668,8118.491463,86047.290239,2936.233768,34384.242622,2470.398607,27742.403016
min,0.000000,0.0000,0.000000,0.000000,60.400000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5%,1.000000,1.0000,18.901667,43.670000,371.166667,0.000000,0.000000,126.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
10%,2.000000,2.0000,20.665000,51.512500,383.316667,0.000000,0.000000,370.200000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,13.500000,11.5000,26.291667,62.118333,473.983333,1.308517,252.000000,3356.320000,0.000000,23205.750000,0.000000,0.000000,0.000000,0.000000
95%,26.000000,22.0000,32.395000,81.269491,670.600000,3.335720,1499.420000,20279.380000,18255.190000,209711.909800,9763.320000,105245.280000,9242.350000,73813.480000
max,27.000000,23.0000,40.000000,81.269491,670.600000,5.462324,1499.420000,42855.230000,18255.190000,292014.969800,9763.320000,200561.760000,9242.350000,190016.450000


In [50]:
traininput = preprocessing('train_input/*.csv')

finish!!


In [51]:
testinput = preprocessing('test_input/*.csv')

finish!!


In [53]:
colname=traininput.columns

In [54]:
colname

Index(['DAT', 'obs_time', '내부온도관측치', '내부습도관측치', 'co2관측치', 'ec관측치', '시간당분무량',
       '일간누적분무량', '시간당백색광량', '일간누적백색광량', '시간당적색광량', '일간누적적색광량', '시간당청색광량',
       '일간누적청색광량'],
      dtype='object')

In [55]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
trainsd = scaler.fit(traininput.iloc[:,2:]).transform(traininput.iloc[:,2:])
testsd = scaler.fit(traininput.iloc[:,2:]).transform(testinput.iloc[:,2:])

trainsd = pd.DataFrame(trainsd,columns=colname[2:]).reset_index(drop=True)
testsd = pd.DataFrame(testsd,columns=colname[2:]).reset_index(drop=True)

train=pd.merge(traininput.iloc[:,:2].reset_index(drop=True),trainsd,left_index=True, right_index=True)
test=pd.merge(testinput.iloc[:,:2].reset_index(drop=True),testsd,left_index=True, right_index=True)

In [28]:
train.iloc[-24*28-672:-24*28]

,DAT,obs_time,내부온도관측치,내부습도관측치,co2관측치,ec관측치,시간당분무량,일간누적분무량,시간당백색광량,일간누적백색광량,시간당적색광량,일간누적적색광량,시간당청색광량,일간누적청색광량,시간당총광량,일간누적총광량
299712,0,0,0.949378,-0.391741,0.155449,-0.511575,-0.962502,-0.922148,-0.775478,-0.847439,-0.535571,-0.529414,-0.502909,-0.509754,-1.002462,-1.080662
299713,0,1,0.889589,-0.493852,0.158968,-0.512258,-0.962502,-0.922148,-0.775478,-0.847439,-0.535571,-0.529414,-0.502909,-0.509754,-1.002462,-1.080662
299714,0,2,0.908424,-0.441351,0.198180,-0.510333,-0.962502,-0.922148,-0.775478,-0.847439,-0.535571,-0.529414,-0.502909,-0.509754,-1.002462,-1.080662
299715,0,3,0.862403,-0.420621,0.213546,-0.508465,0.450708,-0.810781,-0.775478,-0.847439,-0.535571,-0.529414,-0.502909,-0.509754,-1.002462,-1.080662
299716,0,4,0.907157,-0.398746,0.211058,-0.499973,-0.962502,-0.810781,-0.775478,-0.847439,-0.535571,-0.529414,-0.502909,-0.509754,-1.002462,-1.080662
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
300379,27,19,0.000673,-0.445926,-0.434160,-0.299573,-0.962502,-0.553853,-0.773961,0.856657,-0.535571,0.117776,-0.502909,-0.132637,-1.001105,0.761574
300380,27,20,0.332952,-0.477093,-0.428160,-0.304022,-0.730949,-0.535605,-0.775478,0.856657,-0.535571,0.117776,-0.502909,-0.132637,-1.002462,0.761574
300381,27,21,0.445260,-0.448070,-0.491818,-0.309000,-0.962502,-0.535605,-0.775478,0.856657,-0.535571,0.117776,-0.502909,-0.132637,-1.002462,0.761574
300382,27,22,0.621321,-0.498681,-0.529867,-0.314690,-0.962502,-0.535605,-0.775478,0.856657,-0.535571,0.117776,-0.502909,-0.132637,-1.002462,0.761574


In [56]:
def preprocessing2(input_path, data):
    all_input_list = sorted(glob.glob(input_path))
    for i in range(0,len(all_input_list)):
        df = data.iloc[ 24*28*i : 24*28*(i+1)]
        df.to_csv(all_input_list[i],index=False)
    print("완료~!")

In [57]:
preprocessing2('train_input/*.csv',train)

완료~!


In [58]:
preprocessing2('test_input/*.csv',test)

완료~!


In [59]:
def make_dataset(input_path, target_path):
    train = pd.DataFrame()
    all_input_list = sorted(glob.glob(input_path))
    all_target_list = sorted(glob.glob(target_path))
    for x, y in zip(all_input_list,all_target_list):
        x = pd.read_csv(x)
        y = pd.read_csv(y)
        x = x.iloc[:,1:]
        col_list = x.columns[1:]
        for i in range(0,28) :
            day = x.iloc[24*i:24*i+24]

            day_white = day[day['일간누적백색광량']>=0]['일간누적백색광량'].max()
            day_water = day[day['일간누적분무량']>=0]['일간누적분무량'].max()
            day_blue = day[day['일간누적청색광량']>=0]['일간누적청색광량'].max()
            day_red = day[day['일간누적적색광량']>=0]['일간누적적색광량'].max()

            time_list = day['obs_time'].unique()
            for col in col_list :
                if col in ['일간누적백색광량','일간누적청색광량','일간누적적색광량','일간누적분무량']:
                    for t in range(0,len(time_list)):
                        time = time_list[t]
                        value1 = day[day['obs_time']==time][col].iloc[0]
                        x[col+str(time)] = value1
                  
                else :
                    for t in range(0,len(time_list)-3) :
                        time = time_list[t]
                        ntime = time_list[t+1]
                        nntime = time_list[t+2]
                        nnntime = time_list[t+3]
                        value1 = day[day['obs_time']==time][col].iloc[0]
                        value2 = day[day['obs_time']==ntime][col].iloc[0]
                        value3 = day[day['obs_time']==nntime][col].iloc[0]
                        value4 = day[day['obs_time']==nnntime][col].iloc[0]
                        x[col+str(time)+str("+")+str(nnntime)] = (value1 + value2 + value3 + value4)/4

            x['day_water_weight_max'] = (day_water*(i+1))
            x['day_white_weight_max'] = (day_white*(i+1))
            x['day_red_weight_max'] = (day_red*(i+1))
            x['day_blue_weight_max'] = (day_blue*(i+1))

            nx = x.iloc[:1,15:]
            ny = y.iloc[i:i+1].reset_index(drop=True)
            xy = pd.merge(nx,ny,left_index=True, right_index=True)
            train = pd.concat([train,xy]).reset_index(drop=True)

    return train

In [60]:
train_input_path, train_target_path = 'train_input/*.csv', 'train_target/*.csv'
test_input_path, test_target_path = 'test_input/*.csv', 'test_target/*.csv'

train = make_dataset(train_input_path, train_target_path)
test = make_dataset(test_input_path, test_target_path)

In [61]:
train.to_csv('train_20퍼tfmean.csv',index=False)
test.to_csv('test_20퍼tfmean.csv',index=False)

In [64]:
train

,내부온도관측치2+5,내부온도관측치3+6,내부온도관측치4+7,내부온도관측치5+8,내부온도관측치6+9,내부온도관측치7+10,내부온도관측치8+11,내부온도관측치9+12,내부온도관측치10+13,내부온도관측치11+14,...,일간누적청색광량20,일간누적청색광량21,일간누적청색광량22,일간누적청색광량23,day_water_weight_max,day_white_weight_max,day_red_weight_max,day_blue_weight_max,DAT,predicted_weight_g
0,-0.549546,-0.406539,-0.209239,0.004247,0.249080,0.393726,0.489610,0.580373,0.656793,0.735467,...,-0.638521,-0.638521,-0.638521,-0.638521,2.007294,2.086867,NaN,NaN,1,0.167719
1,-0.487672,-0.300616,-0.019314,0.299481,0.642452,0.795088,0.894455,0.977637,1.052623,1.121258,...,-0.638521,-0.638521,-0.638521,-0.638521,4.046866,4.187597,NaN,NaN,2,0.181787
2,-0.360236,-0.174614,0.104844,0.420566,0.693672,0.649827,0.472605,0.258914,0.106483,0.059975,...,-0.638521,-0.638521,-0.638521,-0.638521,6.354284,6.262092,NaN,NaN,3,0.265921
3,-0.591546,-0.398344,-0.120730,0.047272,0.186755,0.127954,0.000108,-0.009317,0.021048,0.026579,...,-0.638521,-0.638521,-0.638521,-0.638521,8.093732,8.375194,NaN,NaN,4,0.423650
4,-0.523731,-0.402032,-0.234029,-0.068690,0.119186,0.207694,0.288622,0.361969,0.440439,0.512967,...,-0.638521,-0.638521,-0.638521,-0.638521,9.928871,10.442459,NaN,NaN,5,0.475272
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
779,-0.373143,-0.189160,0.079234,0.361560,0.680559,0.780131,0.826025,0.904084,0.966163,1.028652,...,-0.638521,-0.638521,-0.638521,-0.638521,45.386705,50.107882,NaN,NaN,24,64.875499
780,-0.364476,-0.186907,0.070834,0.352955,0.666832,0.801644,0.869050,0.926826,0.959197,0.976817,...,-0.638521,-0.638521,-0.638521,-0.638521,50.316834,51.977638,NaN,NaN,25,74.002614
781,-0.376831,-0.218048,0.015516,0.290876,0.591846,0.738746,0.856757,0.991978,1.126380,1.195015,...,-0.638521,-0.638521,-0.638521,-0.638521,51.630130,54.267154,NaN,NaN,26,76.342275
782,-0.177482,0.023506,0.305422,0.595329,0.901216,1.022301,1.066760,1.111014,1.156908,1.188664,...,-0.638521,-0.638521,-0.638521,-0.638521,57.479062,56.271071,NaN,NaN,27,82.621245


In [45]:
pd.read_csv('train_input/CASE_20.csv')

,DAT,obs_time,내부온도관측치,내부습도관측치,co2관측치,ec관측치,시간당분무량,일간누적분무량,시간당백색광량,일간누적백색광량,시간당적색광량,일간누적적색광량,시간당청색광량,일간누적청색광량
0,0,0,25.796666,60.550000,647.433333,2.924978,61.7,61.7,0.0,0.0,0.0000,0.0000,0.000,0.0000
1,0,1,25.285000,62.153333,648.450000,2.922873,61.7,123.4,0.0,0.0,0.0000,0.0000,0.000,0.0000
2,0,2,23.513333,63.510000,633.950000,2.890171,61.7,185.1,0.0,0.0,0.0000,0.0000,0.000,0.0000
3,0,3,22.618333,64.745000,637.100000,2.953415,61.7,246.8,0.0,0.0,0.0000,0.0000,0.000,0.0000
4,0,4,22.951667,62.293333,631.050000,2.968661,61.7,308.5,0.0,0.0,0.0000,0.0000,0.000,0.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
667,27,19,25.880000,74.249168,611.450000,0.000000,61.7,1974.4,0.0,0.0,1244.4096,58379.6892,1178.008,55264.5535
668,27,20,25.928333,76.351667,651.483333,0.000000,61.7,2036.1,0.0,0.0,0.0000,58379.6892,0.000,55264.5535
669,27,21,25.981667,76.766667,670.600000,0.000000,61.7,2097.8,0.0,0.0,0.0000,58379.6892,0.000,55264.5535
670,27,22,25.758333,77.206665,670.600000,0.000000,61.7,2159.5,0.0,0.0,0.0000,58379.6892,0.000,55264.5535
